In [35]:

%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import datetime
import cartopy
import cartopy.crs as ccrs
import pickle
#import Tracking_Functions
from dateutil import relativedelta
import os

import src.Tracking_Functions as Tracking_Functions
from src.TrackingDataLoader import * 
from src.utils import * 
from src.Corrections import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
#data_path = '/work/aa0049/a271109/spice-v2.1/chain/work/polarres_wp3_cmip_CNRM/post/yearly/'

suffix = 'remapped_3x'
file_type = '.nc'
data_path = '/work/aa0238/a271093/data/Jan_runs/CNRM_control_remapped_3x/'
#output_path = '/work/aa0238/a271093/results/MOAAP/IVT_Tracking/CNRM_control_remapped_3x/'
output_path = '/work/aa0238/a271093/scratch/Track_test/'

threshold_file = '/work/aa0238/a271093/data/input/IVT_85_percentiles_CNMR_control_3dx3dy.nc'
first_year = 1985
last_year = 1987

start_date_list, end_date_list = create_datetime_lists(first_year,last_year) 
first_processed_date = start_date_list[0]
last_processed_date = end_date_list[-1]

In [55]:
IVTuLoader.year_start

1985

In [51]:
IVTuLoader.ds

AttributeError: 'TrackingDataLoader' object has no attribute 'ds'

In [ ]:
#ds_ivt_pctl=xr.open_dataset('/work/aa0049/a271109/data_misc/AR_Guan/code/ivt_percentile_mlauer_removed-ens-lev_remapbilWP3domain.nc',decode_times=False)

#IVTtrheshold=ds_ivt_pctl.ivt1.values

ds_ivt_pctl=xr.open_dataset(threshold_file,decode_times=False)
IVTtrheshold=ds_ivt_pctl.IVT_85perc.values

IVTuLoader= TrackingDataLoader('IVTu', data_path, suffix=suffix, file_type = file_type)
IVTvLoader= TrackingDataLoader('IVTv', data_path, suffix=suffix, file_type = file_type)

dict_keys_offset = 0

for start_date, end_date in zip(start_date_list, end_date_list):
    
    print ("\n \n \n \n")
    
    IVTuLoader.start_date = start_date
    IVTvLoader.start_date = start_date
    
    IVTuLoader.end_date = end_date
    IVTvLoader.end_date = end_date

    print (IVTuLoader.start_date, IVTuLoader.end_date)
    

    IVTudata=IVTuLoader.load_datasets(rm_nc=False)
    IVTvdata=IVTvLoader.load_datasets(rm_nc=False)


    rLon = xr.broadcast(IVTudata.rlon, IVTudata.rlat)[0].values.T
    rLat = xr.broadcast(IVTudata.rlon, IVTudata.rlat)[1].values.T

    Lon = xr.broadcast(IVTudata.lon, IVTudata.lat)[0].values
    Lat = xr.broadcast(IVTudata.lon, IVTudata.lat)[1].values

    Mask=1*(rLat>-999)
    Time_sel = IVTuLoader.time_sel
    
    print (Time_sel)
    output_file_name_temp = f'MOAPP_ICON_control_CNRMforc_100and85controlperc_{suffix}'
    output_file_name = f'{output_file_name_temp}_{get_datetime_str(start_date)}-{get_datetime_str(end_date)}'

    

    Tracking_Functions.moaap(Lon = rLon,                           # 2Dlongitude grid centers
                              Lat = rLat,                           # 2D latitude grid spacing
                              Time = Time_sel,                         # datetime vector of data
                              dT = 1,                              # integer - temporal frequency of data [hour]
                              Mask = Mask,                         # mask with dimensions [lat,lon] defining analysis region

                              ivte = IVTudata.IVTu.values, # zonal integrated vapor transport [kg m-1 s-1]
                              ivtn = IVTvdata.IVTv.values, # meidional integrated vapor transport [kg m-1 s-1]
                              regular_Lon = Lon,
                              regular_Lat = Lat,
                              IVTtrheshold = IVTtrheshold, # Integrated water vapor transport threshold for AR detection [kg m-1 s-1]
                               # JLa: additionall fixed threshold 100 in code

                              DataName = output_file_name,
                              OutputFolder=output_path ,
                              dict_keys_offset = dict_keys_offset
                            )
                       
                             
    cleanup_dicts(output_path,
                  output_file_name_temp,
                  start_date,
                  end_date, 
                  last_processed_date,
                  type_='IVT'
                 )
        
    cleanup_dicts(output_path,
                  output_file_name_temp,
                  start_date,
                  end_date, 
                  last_processed_date,
                  type_='ARs'
                 )
    
    correct_nc_file(output_path,
                    output_file_name_temp, 
                    start_date,
                    end_date,
                    last_processed_date
                   )
                             
    dict_keys_offset +=5000


 
 
 

1985-01-01 00:00:00 1985-08-01 00:00:00
/work/aa0238/a271093/data/Jan_runs/CNRM_control_remapped_3x/IVTu/IVTu_1985010100-1985123123_remapped_3x.nc
<xarray.Dataset> Size: 1GB
Dimensions:       (time: 8760, rlat: 194, rlon: 193, nv: 4)
Coordinates:
  * time          (time) datetime64[ns] 70kB 1985-01-01 ... 1985-12-31T23:00:00
    lon           (rlat, rlon) float32 150kB ...
    lat           (rlat, rlon) float32 150kB ...
  * rlon          (rlon) float32 772B -29.9 -29.6 -29.3 -29.0 ... 27.1 27.4 27.7
  * rlat          (rlat) float32 776B -24.2 -23.9 -23.6 -23.3 ... 33.1 33.4 33.7
Dimensions without coordinates: nv
Data variables:
    lon_bnds      (rlat, rlon, nv) float32 599kB ...
    lat_bnds      (rlat, rlon, nv) float32 599kB ...
    rotated_pole  |S1 1B ...
    IVTu          (time, rlat, rlon) float32 1GB ...
Attributes: (12/17)
    CDI:                       Climate Data Interface version 2.0.5 (https://...
    Conventions:               CF-1.4
    source:                

100%|██████████| 715/715 [00:24<00:00, 28.70it/s]


        00:00:30.89
{}
9
            Loop over 1573 objects


/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:247: RuntimeWarning: All-NaN slice encountered
  obj_min = np.nanmin(data_slice, axis=(1, 2))
/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:248: RuntimeWarning: All-NaN slice encountered
  obj_max = np.nanmax(data_slice, axis=(1, 2))
/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:249: RuntimeWarning: Mean of empty slice
  obj_mean = np.nanmean(data_slice, axis=(1, 2))
/work/aa0238/a271093/Conda/MOAAP_env/lib/python3.12/site-packages/scipy/ndimage/_measurements.py:1541: RuntimeWarning: invalid value encountered in scalar divide
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


        check if MSs quallify as ARs


...

/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:247: RuntimeWarning: All-NaN slice encountered
  obj_min = np.nanmin(data_slice, axis=(1, 2))
/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:248: RuntimeWarning: All-NaN slice encountered
  obj_max = np.nanmax(data_slice, axis=(1, 2))
/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:249: RuntimeWarning: Mean of empty slice
  obj_mean = np.nanmean(data_slice, axis=(1, 2))
/work/aa0238/a271093/Conda/MOAAP_env/lib/python3.12/site-packages/scipy/ndimage/_measurements.py:1541: RuntimeWarning: invalid value encountered in scalar divide
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


        check if MSs quallify as ARs


/work/aa0238/a271093/Scripts/MOAAP/src/Tracking_Functions.py:3840: RuntimeWarning: divide by zero encountered in scalar divide
  if DIST.max()/DIST.min() < AR_width_lenght_ratio:


            Loop over 1606 objects
        00:01:21.47
 
Save the object masks into a joint netCDF
Saved: /work/aa0238/a271093/scratch/Track_test/ObjectMasks_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1986_01_01-1986_08_01.nc
        00:00:23.61
clean up /work/aa0238/a271093/scratch/Track_test/IVT_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1986_01_01-1986_07_01.pkl
clean up finished
remove old pickle file
clean up /work/aa0238/a271093/scratch/Track_test/ARs_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1986_01_01-1986_07_01.pkl
clean up finished
remove old pickle file
correct nc-file with 
 cdo -selmon,1,2,3,4,5,6 /work/aa0238/a271093/scratch/Track_test/ObjectMasks_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1986_01_01-1986_08_01.nc /work/aa0238/a271093/scratch/Track_test/ObjectMasks_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1986_01_01-1986_07_01_corrected.nc

 
 
 

1986-07-01 00:00:00 1987-01-01 00:00:0

  0%|          | 1/652 [00:00<01:13,  8.90it/s]

In [ ]:
with open('/work/aa0238/a271093/results/MOAAP/IVT_Tracking/CNRM_control_test/IVT_MOAPP_ICON_control_CNRMforc_100and85controlperc_remapped_3x_1984_01_01-1984_07_01_corrected.pkl', 'rb') as pickle_file:
    IVT1 = pickle.load(pickle_file)
